In [15]:
import numpy as np
import copy

blocks = [np.array([[1, 1, 1], [1, 1, 1]]), 
          np.array([[1], [1], [1], [1]]),
          np.array([[1, 1], [1, 1]])]

board = np.array([[0, 0, 1, 0, 0, 1, 0, 1],
         [0, 0, 1, 0, 1, 1, 0, 1],
         [0, 0, 1, 1, 1, 1, 0, 0],
         [0, 0, 1, 0, 1, 0, 0, 0],
         [0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 0, 0, 1],
         [0, 0, 0, 1, 1, 0, 0, 1],
         [0, 0, 1, 1, 1, 1, 0, 1]])

def permutations(blocks, fi):
    result = []
    if fi == len(blocks) - 1:
        return [blocks[:]]
    for i in range(fi, len(blocks)):
        blocks = swap(blocks, i, fi)
        result.extend(permutations(blocks, fi + 1))
        blocks = swap(blocks, i, fi)
    return result

def swap(blocks, i, fi):
    temp = blocks[i]
    blocks[i] = blocks[fi]
    blocks[fi] = temp
    return blocks

def waysToFit(figure, board):
    firstFit = []
    for rowNumber in range(board.shape[0]):
        # print(board[rowNumber])
        for cellNumber in range(board.shape[1]):
            # print(board[rowNumber][cellNumber])
            cellValue = board[rowNumber][cellNumber]
            if (cellValue == 1 and figure[0][0] == 0) or (cellValue == 0 and figure[0][0] == 1) or (cellValue == 0 and figure[0][0] == 0):
                canFit = True
                if cellNumber + figure.shape[1] > board.shape[0]:
                    canFit = False
                    break
                if rowNumber + figure.shape[0] > board.shape[1]:
                    canFit = False
                    break
                for figureRow in range(figure.shape[0]):
                    for figureColumn in range(figure.shape[1]):
                        if figure[figureRow][figureColumn] == 1:
                            if board[rowNumber + figureRow][cellNumber + figureColumn] == 1:
                                canFit = False
                                break
                if canFit:
                    firstFit.append([figure, rowNumber, cellNumber])
    return firstFit
def updateBoard(board, figure, x, y):
    board = board.copy()
    clearedRow = 0
    for row in range(figure.shape[0]):
        for column in range(figure.shape[1]):
            board[x+row][y+column] += figure[row][column]
    rowsToClear = []
    for row in range(board.shape[0]):
        for column in range(board.shape[1]):
            canClear = True
            if board[row][column] == 0:
                canClear = False
                break
        if canClear:
            rowsToClear.append(row)    
    colsToClear = []
    for column in range(board.shape[1]):
        for row in range(board.shape[0]):
            canClear = True
            if board[row][column] == 0:
                canClear = False
                break
        if canClear:
            colsToClear.append(column)
    for row in range(board.shape[0]):
        for column in range(board.shape[1]):
            if row in rowsToClear or column in colsToClear:
                board[row][column] = 0
    if len(colsToClear) > 0 or len(rowsToClear) > 0:
        clearedRow = len(colsToClear) + len(rowsToClear)
    return board, clearedRow

originalBoard = board.copy()

allPermutations = permutations(blocks, 0)
def tryOutAllMethods(board, permutation, fi):

    workingPaths = []
    pathTaken = []
    blockIndex = 0
    if len(permutation) == 3:
        for _, rowToFit1, colToFit1 in waysToFit(permutation[blockIndex], board):
            board1, clearedRow = updateBoard(board, permutation[blockIndex], rowToFit1, colToFit1)
            pathTaken.append([permutation[blockIndex], rowToFit1, colToFit1, clearedRow])
            for _, rowToFit2, colToFit2 in waysToFit(permutation[blockIndex + 1], board1):
                board2, clearedRow = updateBoard(board1, permutation[blockIndex + 1], rowToFit2, colToFit2)
                if np.array_equal(permutation[blockIndex+1], np.array([[1], [1], [1], [1]])) and rowToFit2 == 3 and colToFit2 == 5:
                    print("Sup")
                pathTaken.append([permutation[blockIndex + 1], rowToFit2, colToFit2, clearedRow])
                for _, rowToFit3, colToFit3 in waysToFit(permutation[blockIndex + 2], board2):
                    board3, clearedRow = updateBoard(board2, permutation[blockIndex + 2], rowToFit3, colToFit3)
                    pathTaken.append([permutation[blockIndex + 2], rowToFit3, colToFit3, clearedRow])
                    pathTaken.append(board3)
                    workingPaths.append(copy.deepcopy(pathTaken))
                    pathTaken = [pathTaken[0], pathTaken[1]]
                pathTaken = [pathTaken[0]]
            pathTaken = []
        return workingPaths
    elif len(permutation) == 2:
        for _, rowToFit1, colToFit1 in waysToFit(permutation[blockIndex], board):
            board1, clearedRow = updateBoard(board, permutation[blockIndex], rowToFit1, colToFit1)
            pathTaken.append([permutation[blockIndex], rowToFit1, colToFit1, clearedRow])
            for _, rowToFit2, colToFit2 in waysToFit(permutation[blockIndex + 1], board1):
                board2, clearedRow = updateBoard(board1, permutation[blockIndex + 1], rowToFit2, colToFit2)
                pathTaken.append([permutation[blockIndex + 1], rowToFit2, colToFit2, clearedRow])
                pathTaken.append(board2)
                workingPaths.append(copy.deepcopy(pathTaken))
                pathTaken = [pathTaken[0]]
            pathTaken=[]
        return workingPaths
    elif len(permutation) == 1:
        for _, rowToFit, colToFit in waysToFit(permutation[blockIndex], board):
            board1, clearedRow = updateBoard(board, permutation[blockIndex], rowToFit, colToFit)
            pathTaken.append([permutation[blockIndex], rowToFit, colToFit, clearedRow])
            pathTaken.append(board1)
            workingPaths.append(copy.deepcopy(pathTaken))
            pathTaken = []
        return workingPaths
allSolutions = []
for index, i in enumerate(allPermutations):
    if index == 3:
        pass
    results = tryOutAllMethods(board, copy.deepcopy(allPermutations[index]), 0)
    if results:
        allSolutions.append(results)
allSolutions

Sup
Sup


[[[[array([[1],
           [1],
           [1],
           [1]]),
    0,
    0,
    0],
   [array([[1, 1],
           [1, 1]]),
    2,
    6,
    1],
   [array([[1, 1, 1],
           [1, 1, 1]]),
    5,
    5,
    0],
   array([[1, 0, 1, 0, 0, 1, 0, 0],
          [1, 0, 1, 0, 1, 1, 0, 0],
          [1, 0, 1, 1, 1, 1, 1, 0],
          [1, 0, 1, 0, 1, 0, 1, 0],
          [0, 0, 1, 1, 1, 1, 1, 0],
          [0, 0, 1, 1, 1, 1, 1, 1],
          [0, 0, 0, 1, 1, 1, 1, 1],
          [0, 0, 1, 1, 1, 1, 0, 0]])],
  [[array([[1],
           [1],
           [1],
           [1]]),
    0,
    0,
    0],
   [array([[1, 1],
           [1, 1]]),
    4,
    0,
    1],
   [array([[1, 1, 1],
           [1, 1, 1]]),
    3,
    5,
    0],
   array([[1, 0, 1, 0, 0, 1, 0, 1],
          [1, 0, 1, 0, 1, 1, 0, 1],
          [1, 0, 1, 1, 1, 1, 0, 0],
          [1, 0, 1, 0, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 1, 1, 1],
          [1, 1, 1, 1, 1, 0, 0, 1],
          [0, 0, 0, 1, 1, 0, 0, 1],
          [0, 0, 1, 1

In [14]:
count = 0
for i in allSolutions:
    for j in i:
        count += 1
        print(count)
        print(j)

1
[[array([[1],
       [1],
       [1],
       [1]]), 0, 0, 0], [array([[1, 1],
       [1, 1]]), 2, 6, 1], [array([[1, 1, 1],
       [1, 1, 1]]), 5, 5, 0], array([[1, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0]])]
2
[[array([[1],
       [1],
       [1],
       [1]]), 0, 0, 0], [array([[1, 1],
       [1, 1]]), 4, 0, 1], [array([[1, 1, 1],
       [1, 1, 1]]), 3, 5, 0], array([[1, 0, 1, 0, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 1]])]
3
[[array([[1],
       [1],
       [1],
       [1]]), 0, 1, 0], [array([[1, 1],
       [1, 1]]), 2, 6, 1], [array([[1, 1, 1],
       [1, 1, 1]]), 5, 5, 0], 